In [1]:
%load_ext autoreload
%autoreload 2

# Import Libraries

In [2]:
import os

from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizer

from dataset import *
from model import *
from trainer import Trainer

In [3]:
PATH = '../'
MAX_LEN = 128
BATCH_SIZE = 64

# Loading data

In [4]:
train_data = pd.read_csv(os.path.join(PATH, 'train.csv'))
test_data = pd.read_csv(os.path.join(PATH, 'test.csv'))

train_data = train_data[train_data["movie_description"].notna()]
train_data.head()

,movie_name,movie_description,target
0,Hellraiser,A new take on Clive Barker's 1987 horror class...,Horror
1,Hocus Pocus 2,It's been 29 years since someone lit the Black...,Kids
2,X,"In 1979, a group of young filmmakers set out t...",Horror
3,Piggy,With the summer sun beating down on her rural ...,Horror
4,Deadstream,After a public controversy left him disgraced ...,Horror


# Encoding labels

In [5]:
le = LabelEncoder()
le.fit(train_data["target"])
train_data["target"] = le.transform(train_data["target"])
train_data.head()

,movie_name,movie_description,target
0,Hellraiser,A new take on Clive Barker's 1987 horror class...,3
1,Hocus Pocus 2,It's been 29 years since someone lit the Black...,4
2,X,"In 1979, a group of young filmmakers set out t...",3
3,Piggy,With the summer sun beating down on her rural ...,3
4,Deadstream,After a public controversy left him disgraced ...,3


In [6]:
train_split, val_split = train_test_split(train_data, train_frac=0.85)

# Loading tokenizer from pretrained

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

# Creating datasets and dataloaders

In [8]:
train_dataset = RottenTomatoesDataset(train_split, tokenizer, MAX_LEN)
val_dataset = RottenTomatoesDataset(val_split, tokenizer, MAX_LEN)
test_dataset = RottenTomatoesDataset(test_data, tokenizer, MAX_LEN)

In [9]:
train_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': BATCH_SIZE,
               'shuffle': False,
               'num_workers': 0
               }

train_dataloader = DataLoader(train_dataset, **train_params)
val_dataloader = DataLoader(val_dataset, **test_params)
test_dataloader = DataLoader(test_dataset, **test_params)


# Loading pretrained model from Huggingface

In [10]:
config = {
    'num_classes': 6,
    'dropout_rate': 0.3
}
model = DistilBertForClassification(
    'distilbert-base-uncased',
    config=config
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Creating Trainer object and fitting the model

In [11]:
trainer_config = {
    "lr": 3e-4,
    "n_epochs": 2,
    "weight_decay": 1e-6,
    "batch_size": 64,
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}
t = Trainer(trainer_config)
t.fit(
    model,
    train_dataloader,
    val_dataloader
)

Epoch 1/2


  0%|          | 0/73 [00:00<?, ?it/s]

/home/limp/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  0%|          | 0/13 [00:00<?, ?it/s]

0.5374395847320557
Epoch 2/2


  0%|          | 0/73 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

0.5591787099838257


DistilBertForClassification(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): L

# Save model

In [12]:
t.save("baseline_model.ckpt")

# Load pretrained Model

In [13]:
t = Trainer.load("baseline_model.ckpt")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Get testset predictions


In [14]:
predictions = t.predict(test_dataloader)

# Create submission


In [15]:
sample_submission = pd.read_csv("../sample_submission.csv")
sample_submission["target"] = le.inverse_transform(predictions)
sample_submission.head()

,target
0,Horror
1,Drama
2,Horror
3,Horror
4,Comedy


In [16]:
sample_submission.to_csv("submission.csv", index=False)

In [17]:
# Accuracy 0.597715